In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('talk')
sns.set_style('white')


![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
oorder = ['Intergenic', 'Antisense', 'Out of Frame','In Frame',  'Annotated', ]
opalette = sns.color_palette("rainbow", 5).as_hex()
#del opalette[4]
genome='MIT9313'

In [5]:
found_sorf_df = pd.read_pickle('found_sorf_ISME16.pkl.gz')

In [6]:
found_sorf_df.shape

(180, 78)

# Gene Families

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(found_sorf_df['product'].value_counts())

hypothetical protein                                                                            66
                                                                                                22
Nif11-like leader peptide family natural product precursor                                      18
high light inducible protein                                                                     4
2Fe-2S iron-sulfur cluster binding domain-containing protein                                     2
DUF4278 domain-containing protein                                                                2
DUF3252 domain-containing protein                                                                1
50S ribosomal protein L27                                                                        1
NAD(P)H-quinone oxidoreductase subunit O                                                         1
photosystem I reaction center subunit PsaK                                                       1
transcript

In [8]:
found_sorf_df['family2'] = found_sorf_df['family']
found_sorf_df.loc[found_sorf_df['product'].str.contains('ribosomal'), 'family2'] = 'ribosomal'
found_sorf_df.loc[found_sorf_df['product'].str.contains('photosystem'), 'family2'] = 'photosystem'
found_sorf_df.loc[found_sorf_df['product'].str.contains('hypothetical'), 'family2'] = 'hypothetical'
found_sorf_df.loc[found_sorf_df['product'] == '', 'family2'] = 'N/A'
found_sorf_df.loc[found_sorf_df['product'].str.contains('domain-containing protein') &
                  found_sorf_df['product'].str.startswith('DUF') , 'family2'] = 'DUF'
found_sorf_df.loc[found_sorf_df['product'].str.contains('family protein') &
                  found_sorf_df['product'].str.startswith('DUF') , 'family2'] = 'DUF'
found_sorf_df.loc[found_sorf_df['product'].str.contains('high light inducible protein'), 'family2'] = 'HLIP'
found_sorf_df.loc[found_sorf_df['family2'].isna(), 'family2'] = 'other'
found_sorf_df.loc[~found_sorf_df['family'].isna(), 'family2'] = found_sorf_df.loc[~found_sorf_df['family'].isna(), 'family']
                                                                     

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 100000):  # more options can be specified also
    #display(found_sorf_df.loc[found_sorf_df.family2.isna(), 'product'].value_counts())    
    display(found_sorf_df.loc[found_sorf_df.family2.isna(), 'product'].sort_values().reset_index(drop=True))

Series([], Name: product, dtype: object)

In [10]:
found_sorf_df.groupby(['family2'])['product'].value_counts()

family2    product                          
CCRG-1     guanylate-binding protein            1
CCRG-2     CCRG-2 family RiPP                   1
           hypothetical protein                 1
DUF        DUF4278 domain-containing protein    2
           DUF1816 domain-containing protein    1
                                               ..
ribosomal  50S ribosomal protein L28            1
           50S ribosomal protein L29            1
           50S ribosomal protein L31            1
           50S ribosomal protein L32            1
           50S ribosomal protein L33            1
Name: product, Length: 73, dtype: int64

In [11]:
found_sorf_df.loc[found_sorf_df['family2'] == 'other']['product'].value_counts()

2Fe-2S iron-sulfur cluster binding domain-containing protein                                    2
Asp-tRNA(Asn)/Glu-tRNA(Gln) amidotransferase subunit GatC                                       1
rubredoxin                                                                                      1
chlororespiratory reduction protein 7                                                           1
ssl1498 family light-harvesting-like protein                                                    1
HU family DNA-binding protein                                                                   1
NifU family protein                                                                             1
BolA family transcriptional regulator                                                           1
glutaredoxin 3                                                                                  1
translation initiation factor IF-1                                                              1
YciI family protein 

# BLAST vs SORFS

In [12]:
def is_in_found_sorf(r):
    orfid = r.id.replace('lcl|','')
    return found_sorf_df.orfid.isin([orfid]).sum() > 0
    

In [13]:
accession = sf.get_accession(genome)
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')

In [14]:
from Bio import SeqIO
records = (r for r in SeqIO.parse(orf_fpath, "fasta") if is_in_found_sorf(r))

In [15]:
out_fpath = os.path.join(orf_dpath, f'{genome}_found_proteome.fasta')
count = SeqIO.write(records, out_fpath, "fasta")
print("Saved %i records from %s to %s" % (count, orf_fpath, out_fpath))

Saved 169 records from orffinder_tmp\BX548175.orffinder.fa to orffinder_tmp\MIT9313_found_proteome.fasta


 mit9313_found_vs_pro_syn    https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8P2ACMPV014    
 mit9313_found_vs_nr         https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8P26YCM7014
 mit9313_found_vs_swisprot   https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=8SHSH1FU014
 

In [48]:
blast_dpath = os.path.join('orffinder_tmp','blast results')
blast_vs_nr_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_nr-Alignment.xml')
blast_vs_pro_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_pro_syn-Alignment.xml')
blastn_vs_pro_dpath = os.path.join(blast_dpath, 'mit9313_blastn_vs_pro_syn-Alignment.xml')

blast_vs_sorf_dpath = os.path.join(blast_dpath, 'mit9313_sorf_all_vs_all.xml')
blast_vs_swisprot_dpath = os.path.join(blast_dpath, 'mit9313_found_vs_swisprot-Alignment.xml')

os.listdir(blast_dpath)

['mit9313_blastn_vs_pro_syn-Alignment.xml',
 'MIT9313_found_proteome.fasta',
 'mit9313_found_vs_nr-Alignment.xml',
 'mit9313_found_vs_pro_syn-Alignment.xml',
 'mit9313_found_vs_swisprot-Alignment.xml',
 'mit9313_sorf_all_vs_all.xml']

In [17]:
from Bio.Blast import NCBIXML


# BLAST vs SORF

In [18]:
def blast_rec_to_dict(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'aligment_id' : alignment.hit_id,
        'alignment_def' : alignment.hit_def,
        
        }
        for alignment in blast_rec.alignments if alignment.hit_id != query_id
    ]
    return results

In [19]:
fh = open(blast_vs_sorf_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [blast_rec_to_dict(b) for b in blast_records]
blast_vs_sorf_df = pd.DataFrame([i for lst in reslist for i in lst])

In [20]:
fh.close()

In [21]:
for query in blast_records:
    for alignment in query.alignments:
        for hsp in alignment.hsps:
            print('coverage', hsp.align_length / query.query_length)
            print('identitiy', hsp.identities/ hsp.align_length)

In [22]:
blast_vs_sorf_df.head()

,query,aligment_id,alignment_def
0,ORF2049_BX548175.1:913137:913382,ORF26372_BX548175.1:2236618:2236379,unnamed protein product
1,ORF2049_BX548175.1:913137:913382,ORF26370_BX548175.1:2237020:2236769,unnamed protein product
2,ORF2049_BX548175.1:913137:913382,ORF20045_BX548175.1:274995:274750,unnamed protein product
3,ORF2049_BX548175.1:913137:913382,ORF20982_BX548175.1:2236277:2236026,unnamed protein product
4,ORF2049_BX548175.1:913137:913382,ORF15639_BX548175.1:2235150:2234896,unnamed protein product


In [23]:
def create_sorfs_cluster(blast_vs_sorf_df, cluster_sorfs):
    new_alignments = set(
        blast_vs_sorf_df.loc[blast_vs_sorf_df['query'].isin(cluster_sorfs), 'aligment_id'].unique()
    )
    new_queries = set(
        blast_vs_sorf_df.loc[blast_vs_sorf_df['aligment_id'].isin(cluster_sorfs), 'query'].unique()
    )
    if (new_alignments <= cluster_sorfs) and (new_queries <= cluster_sorfs):
        return cluster_sorfs
    else:
        new_cluster_sorfs = cluster_sorfs | new_alignments | new_queries
        return create_sorfs_cluster(blast_vs_sorf_df, new_cluster_sorfs)
    
    

In [24]:
def create_sorfs_cluster_sel_root(blast_vs_sorf_df, visited):
    not_visited = blast_vs_sorf_df.loc[~blast_vs_sorf_df['query'].isin(visited), 'query'].unique()
    if len(not_visited):
        return create_sorfs_cluster(blast_vs_sorf_df, {not_visited[0]})
    #else
    not_visited2 = blast_vs_sorf_df.loc[~blast_vs_sorf_df['aligment_id'].isin(visited), 'aligment_id'].unique()
    if len(not_visited2):
        return create_sorfs_cluster(blast_vs_sorf_df, {not_visited2[0]})
    # else
    return None
    


In [25]:
visited = set()
clusters = []
while(True):
    new_cluster = create_sorfs_cluster_sel_root(blast_vs_sorf_df, visited)
    if new_cluster is None:
        break
    visited = visited | new_cluster
    clusters.append(new_cluster)
    

In [26]:
len(clusters)

10

In [27]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,cluster_id,family2
578,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARH...,228,75,...,75,8.89242,6.771,unnamed protein product - [ORF533_BX548175.1:2...,4.305928,True,True,True,NaN,other
2285,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDL...,246,81,...,81,8.50516,4.48779,unnamed protein product - [ORF2049_BX548175.1:...,5.280437,True,True,True,1.0,ProcA
2316,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2317,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2380,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVL...,252,83,...,83,9.29551,6.50732,unnamed protein product - [ORF2106_BX548175.1:...,3.996620,True,True,True,NaN,other


In [28]:
for i,c in enumerate(clusters, start=1):
    found_sorf_df.loc[found_sorf_df.orfid.isin(c), 'cluster_id'] = i

In [29]:
found_sorf_df.head()

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,cluster_id,family2
578,ORF533_BX548175.1:237054:237281,BX548175.1,237054,237281,237054,237281,+,MLYLVAPAGDGRDVYATLYAQRMFFLVTLQPRGAQFEVIPYQDARH...,228,75,...,75,8.89242,6.771,unnamed protein product - [ORF533_BX548175.1:2...,4.305928,True,True,True,NaN,other
2285,ORF2049_BX548175.1:913137:913382,BX548175.1,913137,913382,913137,913382,+,MSEEQLKAFIAKVQGDTSLQEQLKAEGADVVAIAKAAGFSITTEDL...,246,81,...,81,8.50516,4.48779,unnamed protein product - [ORF2049_BX548175.1:...,5.280437,True,True,True,1.0,ProcA
2316,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2317,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2380,ORF2106_BX548175.1:940263:940514,BX548175.1,940263,940514,940263,940514,+,MGFLHELEISDGGHRFELASHDHPDHHHLVCVRCGRTEEFESKPVL...,252,83,...,83,9.29551,6.50732,unnamed protein product - [ORF2106_BX548175.1:...,3.996620,True,True,True,NaN,other


In [30]:
found_sorf_df.groupby(['cluster_id','family2'])['product'].value_counts()

cluster_id  family2       product                                                   
1.0         ProcA         Nif11-like leader peptide family natural product precursor    18
                          lantipeptide                                                   1
2.0         N/A                                                                          2
            hypothetical  hypothetical protein                                           1
3.0         HLIP          high light inducible protein                                   4
            other         ssl1498 family light-harvesting-like protein                   1
4.0         hypothetical  hypothetical protein                                           2
5.0         hypothetical  hypothetical protein                                           2
6.0         hypothetical  hypothetical protein                                           2
7.0         hypothetical  hypothetical protein                                           2
8.0  

In [31]:
found_sorf_df.groupby(['cluster_id', 'otype', 'orfid', 'product', #'family'
                      ])['family'].unique().str.join(',').reset_index()

,cluster_id,otype,orfid,product,family
0,1.0,Annotated,ORF15620_BX548175.1:2243295:2243047,Nif11-like leader peptide family natural produ...,ProcA
1,1.0,Annotated,ORF15637_BX548175.1:2235903:2235652,Nif11-like leader peptide family natural produ...,ProcA
2,1.0,Annotated,ORF15638_BX548175.1:2235486:2235229,lantipeptide,ProcA
3,1.0,Annotated,ORF15639_BX548175.1:2235150:2234896,Nif11-like leader peptide family natural produ...,ProcA
4,1.0,Annotated,ORF20045_BX548175.1:274995:274750,Nif11-like leader peptide family natural produ...,ProcA
5,1.0,Annotated,ORF20053_BX548175.1:273630:273355,Nif11-like leader peptide family natural produ...,ProcA
6,1.0,Annotated,ORF2049_BX548175.1:913137:913382,Nif11-like leader peptide family natural produ...,ProcA
7,1.0,Annotated,ORF20960_BX548175.1:2244944:2244663,Nif11-like leader peptide family natural produ...,ProcA
8,1.0,Annotated,ORF20982_BX548175.1:2236277:2236026,Nif11-like leader peptide family natural produ...,ProcA
9,1.0,Annotated,ORF25353_BX548175.1:276446:276159,Nif11-like leader peptide family natural produ...,ProcA


In [32]:
found_sorf_df.loc[found_sorf_df.cluster_id.isna()]['otype'].value_counts()

Annotated       95
Intergenic      20
Out of Frame    17
In Frame         4
Antisense        2
Name: otype, dtype: int64

In [33]:
found_sorf_df.pivot_table(columns='otype', index='family2',aggfunc='count', values='orfid')

otype,Annotated,Antisense,In Frame,Intergenic,Out of Frame
family2,,,,,
CCRG-1,1.0,NaN,NaN,NaN,NaN
CCRG-2,2.0,NaN,NaN,NaN,NaN
DUF,12.0,NaN,2.0,NaN,3.0
HLIP,3.0,NaN,NaN,NaN,1.0
N/A,NaN,NaN,NaN,22.0,NaN
ProcA,16.0,NaN,2.0,NaN,1.0
hypothetical,53.0,2.0,1.0,NaN,9.0
other,24.0,NaN,1.0,NaN,7.0
photosystem,5.0,NaN,1.0,NaN,NaN


In [34]:
found_sorf_df.otype.value_counts()

Annotated       127
Intergenic       22
Out of Frame     22
In Frame          7
Antisense         2
Name: otype, dtype: int64

In [35]:
found_sorf_df.loc[found_sorf_df.otype == 'Intergenic']

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,cluster_id,family2
2316,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2317,ORF2068_BX548175.1:922530:922661,BX548175.1,922530,922661,922530,922661,+,MMGKTKQHKKLVSWAEKAELCATREEAQKLIRKAEKAHAKLES,132,43,...,43,4.93366,9.71533,unnamed protein product - [ORF2068_BX548175.1:...,3.770810,False,True,True,NaN,N/A
2646,ORF2321_BX548175.1:1030986:1031147,BX548175.1,1030986,1031147,1030986,1031147,+,MLSKSITGMFGRRNKKDAPKEEPTETDLERGYRLAAAKQKEILKEL...,162,53,...,53,6.04028,9.89111,unnamed protein product - [ORF2321_BX548175.1:...,5.131342,True,True,True,2.0,N/A
2647,ORF2321_BX548175.1:1030986:1031147,BX548175.1,1030986,1031147,1030986,1031147,+,MLSKSITGMFGRRNKKDAPKEEPTETDLERGYRLAAAKQKEILKEL...,162,53,...,53,6.04028,9.89111,unnamed protein product - [ORF2321_BX548175.1:...,5.131342,True,True,True,2.0,N/A
8348,ORF7466_BX548175.1:1097200:1097379,BX548175.1,1097200,1097379,1097200,1097379,+,MAGLMSRHGYTISYRDAQSHRCEFCAYANDSFEARQVAMETIKFVH...,180,59,...,59,6.79618,7.02002,unnamed protein product - [ORF7466_BX548175.1:...,4.313464,False,True,True,NaN,N/A
8349,ORF7466_BX548175.1:1097200:1097379,BX548175.1,1097200,1097379,1097200,1097379,+,MAGLMSRHGYTISYRDAQSHRCEFCAYANDSFEARQVAMETIKFVH...,180,59,...,59,6.79618,7.02002,unnamed protein product - [ORF7466_BX548175.1:...,4.313464,False,True,True,NaN,N/A
15314,ORF13811_BX548175.1:1709954:1710130,BX548175.1,1709954,1710130,1709954,1710130,+,MTTLRRFVAITPLAGAIILPLVVPLSMARLGVGAGVLMTLMVSTIW...,177,58,...,58,6.23245,12.0005,unnamed protein product - [ORF13811_BX548175.1...,4.203436,False,True,True,NaN,N/A
15315,ORF13811_BX548175.1:1709954:1710130,BX548175.1,1709954,1710130,1709954,1710130,+,MTTLRRFVAITPLAGAIILPLVVPLSMARLGVGAGVLMTLMVSTIW...,177,58,...,58,6.23245,12.0005,unnamed protein product - [ORF13811_BX548175.1...,4.203436,False,True,True,NaN,N/A
18565,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A
18566,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A


In [36]:
found_sorf_df['Score'] = pd.to_numeric(found_sorf_df['Score'])
found_sorf_df['Coverage'] = pd.to_numeric(found_sorf_df['Coverage'])


In [37]:
found_sorf_df.loc[found_sorf_df.otype == 'Intergenic'].nlargest(10, 'Score')

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,cluster_id,family2
18565,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A
18566,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A
2646,ORF2321_BX548175.1:1030986:1031147,BX548175.1,1030986,1031147,1030986,1031147,+,MLSKSITGMFGRRNKKDAPKEEPTETDLERGYRLAAAKQKEILKEL...,162,53,...,53,6.04028,9.89111,unnamed protein product - [ORF2321_BX548175.1:...,5.131342,True,True,True,2.0,N/A
2647,ORF2321_BX548175.1:1030986:1031147,BX548175.1,1030986,1031147,1030986,1031147,+,MLSKSITGMFGRRNKKDAPKEEPTETDLERGYRLAAAKQKEILKEL...,162,53,...,53,6.04028,9.89111,unnamed protein product - [ORF2321_BX548175.1:...,5.131342,True,True,True,2.0,N/A
24933,ORF22713_BX548175.1:1476551:1476351,BX548175.1,1476351,1476551,1476551,1476351,-,MGKQERRARFKAMTSSERKALIRKKMKAAGLVEGSGVPGTTLSSYD...,201,66,...,66,7.45887,10.1108,unnamed protein product - [ORF22713_BX548175.1...,6.754949,True,True,True,NaN,N/A
24934,ORF22713_BX548175.1:1476551:1476351,BX548175.1,1476351,1476551,1476551,1476351,-,MGKQERRARFKAMTSSERKALIRKKMKAAGLVEGSGVPGTTLSSYD...,201,66,...,66,7.45887,10.1108,unnamed protein product - [ORF22713_BX548175.1...,6.754949,True,True,True,NaN,N/A
33659,ORF30529_BX548175.1:408367:408233,BX548175.1,408233,408367,408367,408233,-,MEWQTNGELARGDLFELVRRLRDVELPSHSNELWRLGAKYEETE,135,44,...,44,5.27162,4.77979,unnamed protein product - [ORF30529_BX548175.1...,3.659508,True,True,True,NaN,N/A
33660,ORF30529_BX548175.1:408367:408233,BX548175.1,408233,408367,408367,408233,-,MEWQTNGELARGDLFELVRRLRDVELPSHSNELWRLGAKYEETE,135,44,...,44,5.27162,4.77979,unnamed protein product - [ORF30529_BX548175.1...,3.659508,True,True,True,NaN,N/A
34136,ORF30974_BX548175.1:185020:184811,BX548175.1,184811,185020,185020,184811,-,MMPNTPTIQAALHELNKQMNDHCLKDIIINGNIGLLRNDNPMEYTE...,210,69,...,69,8.03299,4.99561,unnamed protein product - [ORF30974_BX548175.1...,3.797060,False,True,True,NaN,N/A
34137,ORF30974_BX548175.1:185020:184811,BX548175.1,184811,185020,185020,184811,-,MMPNTPTIQAALHELNKQMNDHCLKDIIINGNIGLLRNDNPMEYTE...,210,69,...,69,8.03299,4.99561,unnamed protein product - [ORF30974_BX548175.1...,3.797060,False,True,True,NaN,N/A


In [38]:
found_sorf_df.loc[found_sorf_df.otype == 'Intergenic'].nlargest(10, 'Coverage')

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,AAs,MW [kDa],calc. pI,tempaccession,max_xCorr,multiple unique peptides,has unique peptides,ms_detected,cluster_id,family2
22031,ORF19962_BX548175.1:317208:317134,BX548175.1,317134,317208,317208,317134,-,MIMRCDDNLNQTLLEEMKELGERQ,75,24,...,24,2.90836,4.43701,unnamed protein product - [ORF19962_BX548175.1...,5.217564,False,True,True,NaN,N/A
22032,ORF19962_BX548175.1:317208:317134,BX548175.1,317134,317208,317208,317134,-,MIMRCDDNLNQTLLEEMKELGERQ,75,24,...,24,2.90836,4.43701,unnamed protein product - [ORF19962_BX548175.1...,5.217564,False,True,True,NaN,N/A
18565,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A
18566,ORF16879_BX548175.1:1658121:1657948,BX548175.1,1657948,1658121,1658121,1657948,-,MMTDQKRDAESPLARFINWVKGVMGGMGTAVVSEKSKNVPPNIGAQ...,174,57,...,57,6.19626,9.99365,unnamed protein product - [ORF16879_BX548175.1...,4.187279,True,True,True,NaN,N/A
33659,ORF30529_BX548175.1:408367:408233,BX548175.1,408233,408367,408367,408233,-,MEWQTNGELARGDLFELVRRLRDVELPSHSNELWRLGAKYEETE,135,44,...,44,5.27162,4.77979,unnamed protein product - [ORF30529_BX548175.1...,3.659508,True,True,True,NaN,N/A
33660,ORF30529_BX548175.1:408367:408233,BX548175.1,408233,408367,408367,408233,-,MEWQTNGELARGDLFELVRRLRDVELPSHSNELWRLGAKYEETE,135,44,...,44,5.27162,4.77979,unnamed protein product - [ORF30529_BX548175.1...,3.659508,True,True,True,NaN,N/A
24933,ORF22713_BX548175.1:1476551:1476351,BX548175.1,1476351,1476551,1476551,1476351,-,MGKQERRARFKAMTSSERKALIRKKMKAAGLVEGSGVPGTTLSSYD...,201,66,...,66,7.45887,10.1108,unnamed protein product - [ORF22713_BX548175.1...,6.754949,True,True,True,NaN,N/A
24934,ORF22713_BX548175.1:1476551:1476351,BX548175.1,1476351,1476551,1476551,1476351,-,MGKQERRARFKAMTSSERKALIRKKMKAAGLVEGSGVPGTTLSSYD...,201,66,...,66,7.45887,10.1108,unnamed protein product - [ORF22713_BX548175.1...,6.754949,True,True,True,NaN,N/A
33096,ORF30023_BX548175.1:644083:643928,BX548175.1,643928,644083,644083,643928,-,MAKRRNLKKEKQERNRSYARKFKKRKLRNDGRGEGAGNGVTGTANN...,156,51,...,51,5.65901,11.3267,unnamed protein product - [ORF30023_BX548175.1...,4.615609,False,True,True,NaN,N/A
33097,ORF30023_BX548175.1:644083:643928,BX548175.1,643928,644083,644083,643928,-,MAKRRNLKKEKQERNRSYARKFKKRKLRNDGRGEGAGNGVTGTANN...,156,51,...,51,5.65901,11.3267,unnamed protein product - [ORF30023_BX548175.1...,4.615609,False,True,True,NaN,N/A


# BLAST vs swisprot

In [59]:
def pro_blastn_to_description(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'pro_syn' : a.hit_def,
        'pro_syn_hsps' : len(a.hsps),
        }
        for a in blast_rec.alignments
    ]
    return results

In [63]:
fh = open(blastn_vs_pro_dpath)
blast_records = NCBIXML.parse(fh)
#reslist = [pro_blastn_to_description(b) for b in blast_records]
#blastn_vs_pro_df = pd.DataFrame([i for lst in reslist for i in lst])

In [64]:
while(True):
    b= next(blast_records)
    if len(b.alignments):
        break

In [66]:
b.alignments[2].hsps

In [61]:
fh.close()

In [62]:
blastn_vs_pro_df.head()

,query,pro_syn,pro_syn_hsps
0,ORF395_BX548175.1:178002:178226,Prochlorococcus marinus MIT9313 complete genome,1
1,ORF395_BX548175.1:178002:178226,"Prochlorococcus marinus str. MIT 9303, complet...",1
2,ORF395_BX548175.1:178002:178226,"Synechococcus sp. KORDI-100, complete genome",1
3,ORF395_BX548175.1:178002:178226,Synechococcus sp. WH 8020 genome,1
4,ORF395_BX548175.1:178002:178226,"Synechococcus sp. CC9311, complete genome",1


In [43]:
found_sorf_df = pd.merge(found_sorf_df, 
         blast_vs_swis_df.loc[blast_vs_swis_df.groupby('query').swis_e.idxmin()].reset_index(drop=True),
         left_on='orfid', right_on='query', how='left')

In [44]:
found_sorf_df.dropna(subset=['swis_description'])[['family2', 'swis_description', 'product']].sort_values('family2')

,family2,swis_description,product
125,DUF,Uncharacterized protein ssl0461 [Synechocystis...,DUF2555 domain-containing protein
124,DUF,"NAD(P)H-quinone oxidoreductase subunit S, chlo...",DUF3252 domain-containing protein
15,HLIP,Uncharacterized protein ycf17; AltName: Full=O...,high light inducible protein
96,HLIP,Uncharacterized protein ycf17; AltName: Full=O...,high light inducible protein
4,other,Ferric uptake regulation protein; Short=Ferric...,transcriptional repressor
171,other,Uncharacterized protein HI_0828 [Haemophilus i...,YciI family protein
166,other,ATP-dependent Clp protease adapter protein Clp...,ATP-dependent Clp protease adapter ClpS
156,other,Ferredoxin [Mastigocladus laminosus],2Fe-2S iron-sulfur cluster binding domain-cont...
133,other,Putative pterin-4-alpha-carbinolamine dehydrat...,pterin-4-alpha-carbinolamine dehydratase
132,other,Ethanolamine utilization protein EutN [Salmone...,carboxysome peptide A


In [45]:
found_sorf_df.dropna(subset=['swis_description'])[['family2','otype', 'swis_description', 'product']]

,family2,otype,swis_description,product
4,other,Out of Frame,Ferric uptake regulation protein; Short=Ferric...,transcriptional repressor
10,other,In Frame,Methylenetetrahydrofolate--tRNA-(uracil-5-)-me...,FADH(2)-oxidizing methylenetetrahydrofolate--t...
11,other,Annotated,Uncharacterized protein ssr3122 [Synechocystis...,BolA family transcriptional regulator
15,HLIP,Out of Frame,Uncharacterized protein ycf17; AltName: Full=O...,high light inducible protein
21,ribosomal,Annotated,30S ribosomal protein S19 [Prochlorococcus mar...,30S ribosomal protein S19
22,ribosomal,Annotated,50S ribosomal protein L29 [Prochlorococcus mar...,50S ribosomal protein L29
26,other,Annotated,Oxygen regulatory protein NreC; AltName: Full=...,response regulator transcription factor
38,ribosomal,Annotated,30S ribosomal protein S15 [Prochlorococcus mar...,30S ribosomal protein S15
39,ribosomal,Annotated,30S ribosomal protein S18 [Prochlorococcus mar...,30S ribosomal protein S18
40,ribosomal,Annotated,50S ribosomal protein L32 [Prochlorococcus mar...,50S ribosomal protein L32


In [46]:
found_sorf_df.otype.value_counts()

Annotated       127
Intergenic       22
Out of Frame     22
In Frame          7
Antisense         2
Name: otype, dtype: int64

In [47]:
found_sorf_df.dropna(subset=['swis_description']).otype.value_counts()

Annotated       35
Out of Frame     3
In Frame         3
Name: otype, dtype: int64

# BLAST vs PRO/SYN

In [39]:
def swisprot_blast_to_description(blast_rec):
    query_id = blast_rec.query.replace(' unnamed protein product','').replace('lcl|','')
    results = [{
        'query' : query_id,
        'swis_description' : d.title.split('=', maxsplit=1)[-1],
        'swis_e' : d.e,
        }
        for d in blast_rec.descriptions
    ]
    return results

In [40]:
fh = open(blast_vs_pro_dpath)
blast_records = NCBIXML.parse(fh)
reslist = [swisprot_blast_to_description(b) for b in blast_records]
blast_vs_swis_df = pd.DataFrame([i for lst in reslist for i in lst])

In [41]:
fh.close()